In [14]:
import pandas as pd
import chromedriver_binary
import time
from selenium import webdriver

### google search with a keyword

In [2]:
driver = webdriver.Chrome() # setup a driver for chrome
driver.get('https://www.google.com/') # open a web page

time.sleep(5)

search_box = driver.find_element_by_name("q")
search_box.send_keys("hello world")
search_box.submit()

time.sleep(5)

driver.close() # close a web page
driver.quit() # close an entire browser

### google検索結果のタイトルとURLの一覧を取得する関数

In [97]:
def ranking(driver):
    i = 1               # ループ番号、ページ番号を定義
    i_max = 5          # 最大何ページまで分析するかを定義
    title_list = []     # タイトルを格納する空リストを用意
    link_list = []      # URLを格納する空リストを用意
    
    # 現在のページが指定した最大分析ページを超えるまでループする
    while i <= i_max:
        # タイトルとリンクはclass="r"に入っている
        class_group = driver.find_elements_by_class_name('r')
        # タイトルとリンクを抽出しリストに追加するforループ
        for elem in class_group:
            title_list.append(elem.find_element_by_class_name('LC20lb').text)           #タイトル(class="LC20lb")
            link_list.append(elem.find_element_by_tag_name("a").get_attribute('href'))  #リンク(aタグのhref属性)

        # 「次へ」は1つしかないが、あえてelementsで複数検索。空のリストであれば最終ページの意味になる。
        if driver.find_elements_by_id('pnnext') == []:
            i = i_max + 1
        else:
            # 次ページのURLはid="pnnext"のhref属性
            next_page = driver.find_element_by_id('pnnext').get_attribute('href')
            print(f"moving to {next_page}...")
            driver.get(next_page)   # 次ページへ遷移する
            i = i + 1               # iを更新
            time.sleep(3)           # 3秒間待機
    
    print("\n")
    
    return title_list, link_list    # タイトルとリンクのリストを戻り値に指定

In [100]:
driver = webdriver.Chrome()
driver.get('https://www.google.com/')

search_box = driver.find_element_by_name("q")
search_box.send_keys("hello")
search_box.submit()

results = ranking(driver)
driver.quit()

h_df = pd.DataFrame(results).T
hello_df = h_df.rename(columns = {0: "title", 1: "URL"})
hello_df

### Xpathで要素指定してスクレイピング

In [104]:
driver = webdriver.Chrome()
driver.get("https://watlab-blog.com/2019/08/13/chrome-xpathhelper/")

xpath = "/html/body/div[3]/div/div[1]/div/article/section/p[39]"
elems = driver.find_elements_by_xpath(xpath)

for elem in elems:
    print(elem.text)
    
driver.quit()

元の文章には色変更や強調のHTMLタグがふんだんに使われていましたが、見事に文章だけが抽出され、その他は削ぎ落されていますね。これぞスクレイピング！


### google検索結果からクラス名でスクレイピング

In [125]:
driver = webdriver.Chrome()
driver.get("https://www.google.com/")

time.sleep(3)

search_box = driver.find_element_by_name("q")
search_box.send_keys('abc')
search_box.submit()

class_name = 'LC20lb'                   # class属性名
class_elems = driver.find_elements_by_class_name(class_name) # classでの指定

# 取得した要素を1つずつ表示
for elem in class_elems:
    print(elem.text)
    
driver.quit()

ABC Home Page - ABC.com
ABC News – Breaking News, Latest News, Headlines & Videos
ABC (Australian Broadcasting Corporation)
ABC News (Australian Broadcasting Corporation)
ABC - YouTube
American Broadcasting Company - Wikipedia
ABC - Home | Facebook
ABC – Live TV & Full Episodes on the App Store





### 検索ワード送って動的にできるテーブルをスクレイピング

In [167]:
# scraping!
def scraper(driver):
    cell_tg = "mat-cell"
    cells = driver.find_elements_by_tag_name(cell_tg)

    # return the results' innerText (.text)
    return [v.text for v in cells]

In [165]:
# desired page
page = "https://www.adrforum.com/domain-dispute/search-decisions"
# full xpath of search box for domain
xpath_q = "/html/body/app-root/div/mat-sidenav-container/mat-sidenav-content/div/app-domain-dispute-container/div/app-search-decisions/div/div/div/div/form/div/mat-card/mat-card-content/div/div[1]/mat-form-field[4]/div/div[1]/div/input"

results = []

# setup a driver
driver = webdriver.Chrome()

# visit the desired page
driver.get(page)

# find a search box, then send a query word
domain_search = driver.find_element_by_xpath(xpath_q)
domain_search.send_keys(".tv")
domain_search.submit()

# wait for the page to render...
time.sleep(15)

# scrape until the very last page
#while ():
results = results + scraper(driver)
    
# waiting for ... what?
time.sleep(3)

# quit
driver.quit()

In [166]:
cols = ["case num", "domains", "case name", "ruleset", "commenced", "status", "decision date"]
N = len(cols)
R = int(len(results)/N)
df = pd.DataFrame([], columns = cols)

for r in range(R):
    i = r*N
    c = i + N
    addRow = pd.DataFrame(results[i:c], index = df.columns).T
    df = df.append(addRow)

df

,case num,domains,case name,ruleset,commenced,status,decision date
0,95897,hollywoodnetwork.tv,"Hollywood Network, Inc, v Video Citizen Network",UDRP,11/15/2000,Transferred,12/21/2000
0,96260,enrononline.tv,Enron Corp. v Kirk Merrikh,UDRP,12/15/2000,Transferred,01/10/2001
0,96261,"bloombergmoney.tv, bloombergnews.tv",Bloomberg L.P. v G. Sandhu,UDRP,12/29/2000,Transferred,02/12/2001
0,96464,premiere.tv,Hachette Filipacchi Presse v Raffledata,UDRP,02/05/2001,Transferred,03/09/2001
0,96557,BESTWESTERN.TV,"Best Western Internat'l., Inc. v Walter Wieczorek",UDRP,02/16/2001,Transferred,03/27/2001
0,96681,"bloombergradio.tv, thebloomberg.tv",Bloomberg L.P. v Martin Aronsky,UDRP,02/28/2001,Transferred,04/11/2001
0,97131,tele7jours.tv,Hachette Filipacchi Presse v Bensadoun Fabien,UDRP,05/10/2001,Transferred,06/13/2001
0,97181,volkswagen.tv,"Volkswagen of America, Inc. v Walter Wieczorek",UDRP,05/30/2001,Transferred,06/26/2001
0,97291,foreverliving.tv,"Aloe Vera of America, Inc. v Adam Norins",UDRP,05/23/2001,Transferred,07/03/2001
0,97849,truevalue.tv,TruServ Corporation v CreativeV,UDRP,07/16/2001,Transferred,08/17/2001
